# Importing Libraries and Setup

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai
import gradio as gr
from IPython.display import display, update_display, Markdown
import google.generativeai

In [2]:
import subprocess
import sys
import io

In [3]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
claude_api_key = os.getenv('ANTHROPIC_API_KEY')
gemini_api_key = os.getenv('GOOGLE_API_KEY')

In [4]:
openai= OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

In [5]:
claude_model = 'claude-sonnet-4-20250514'
openai_model = 'gpt-5-2025-08-07'
gemini_model = 'gemini-2.5-pro'

In [6]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M4 Mac Pro. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."
system_message += 'Keep implementations of random number generators identical so that results match perfectly.'

In [7]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [8]:
def messages_for(python):
    messages = [
        {'role':'system','content':system_message},
        {'role':'user','content':user_prompt_for(python)}
    ]
    return messages

In [9]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized_practice04", "w") as f:
        f.write(code)

# Claude and OpenAI Python --> cpp

In [10]:
def optimize_gpt(python):
    response = openai.chat.completions.create(model = openai_model,messages = messages_for(python),stream = False)
    result = response.choices[0].message.content
    result = result.replace('```','').replace('markdown','')
    write_output(result)
    display(Markdown(result))

In [11]:
def optimize_claude(python):
    response = claude.messages.stream(
        model = claude_model,
        max_tokens = 1000,
        messages = [{'role':'user','content':user_prompt_for(python)}],
        system = system_message
    )
    
    result = ''
    with response as stream:
        for text in stream.text_stream:
            result += text
            print(text, end = '', flush = True)
    write_output(result)

In [12]:
s1 = """
print('this is a string')
"""

In [13]:
exec(s1)

this is a string


In [13]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [15]:
exec(pi)

Result: 3.141592658589
Execution Time: 5.412465 seconds


In [26]:
optimize_gpt(pi)

#include <iostream>
#include <iomanip>
#include <chrono>
#include <cstdint>

static inline double calculate(uint64_t iterations, int param1, int param2) {
    double result = 1.0;
    const double p1 = static_cast<double>(param1);
    const double p2 = static_cast<double>(param2);
    double fi = p1;
    for (uint64_t i = 1; i <= iterations; ++i, fi += p1) {
        double j = fi - p2;
        result -= 1.0 / j;
        j = fi + p2;
        result += 1.0 / j;
    }
    return result;
}

int main() {
    using namespace std::chrono;
    auto start_time = high_resolution_clock::now();
    double result = calculate(100000000ULL, 4, 1) * 4.0;
    auto end_time = high_resolution_clock::now();
    double elapsed = duration<double>(end_time - start_time).count();

    std::cout.setf(std::ios::fixed);
    std::cout << "Result: " << std::setprecision(12) << result << '\n';
    std::cout << "Execution Time: " << std::setprecision(6) << elapsed << " seconds\n";
    return 0;
}

In [22]:
optimize_claude(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        long long j1 = i * param1 - param2;
        long long j2 = i * param1 + param2;
        result += (1.0 / j2) - (1.0 / j1);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000LL, 4, 1) * 4.0;
    auto end_time = std::chrono::high_resolution_clock::now();
    
    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);
    double execution_time = duration.count() / 1000000.0;
    
    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
    std::cout << std::fixed << std::setprecision(6) << "Execution Time: " << execution_time << " seconds" << std::endl;
    
    return 0;
}
```

In [33]:
# Compile C++ and run the executable

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized_practice03_gpt.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.131795 seconds


In [32]:
# Compile C++ and run the executable

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized_practice03_claude.cpp
!./optimized

Result: 3.141592662604
Execution Time: 0.092037 seconds


In [14]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [65]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 14.942410 seconds


In [36]:
optimize_gpt(python_hard)

#include <iostream>
#include <cstdint>
#include <chrono>
#include <iomanip>
#include <string>
#include <algorithm>

static inline uint32_t lcg_next(uint32_t& v) {
    v = v * 1664525u + 1013904223u; // modulo 2^32 via uint32_t wrap
    return v;
}

static std::string int128_to_string(__int128 x) {
    if (x == 0) return "0";
    bool neg = x < 0;
    if (neg) x = -x;
    std::string s;
    while (x > 0) {
        int d = (int)(x % 10);
        s.push_back(char('0' + d));
        x /= 10;
    }
    if (neg) s.push_back('-');
    std::reverse(s.begin(), s.end());
    return s;
}

static __int128 max_subarray_sum(std::size_t n, uint32_t seed, long long min_val, long long max_val) {
    uint32_t state = seed;
    unsigned __int128 urange = (unsigned __int128)((__int128)max_val - (__int128)min_val + 1);
    __int128 best = 0, current = 0;
    bool first = true;
    for (std::size_t i = 0; i < n; ++i) {
        uint32_t rnd = lcg_next(state);
        unsigned __int128 rem = (unsigned __int128)rnd % urange;
        __int128 val = (__int128)rem + (__int128)min_val;
        if (first) {
            current = best = val;
            first = false;
        } else {
            __int128 sum = current + val;
            current = (sum > val) ? sum : val;
            if (current > best) best = current;
        }
    }
    return best;
}

static __int128 total_max_subarray_sum(std::size_t n, uint32_t initial_seed, long long min_val, long long max_val) {
    __int128 total = 0;
    uint32_t seed_state = initial_seed;
    for (int k = 0; k < 20; ++k) {
        uint32_t run_seed = lcg_next(seed_state);
        total += max_subarray_sum(n, run_seed, min_val, max_val);
    }
    return total;
}

int main() {
    std::ios::sync_with_stdio(false);
    std::cin.tie(nullptr);

    std::size_t n = 10000;
    uint32_t initial_seed = 42u;
    long long min_val = -10;
    long long max_val = 10;

    auto start = std::chrono::high_resolution_clock::now();
    __int128 result = total_max_subarray_sum(n, initial_seed, min_val, max_val);
    auto end = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> elapsed = end - start;

    std::cout << "Total Maximum Subarray Sum (20 runs): " << int128_to_string(result) << '\n';
    std::cout << "Execution Time: " << std::fixed << std::setprecision(6) << elapsed.count() << " seconds\n";
    return 0;
}

In [37]:
optimize_claude(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <limits>
#include <chrono>
#include <iomanip>

class LCG {
private:
    uint64_t value;
    static constexpr uint64_t a = 1664525;
    static constexpr uint64_t c = 1013904223;
    static constexpr uint64_t m = 4294967296ULL; // 2^32

public:
    LCG(uint64_t seed) : value(seed) {}
    
    uint64_t next() {
        value = (a * value + c) % m;
        return value;
    }
};

int64_t max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> random_numbers;
    random_numbers.reserve(n);
    
    int range = max_val - min_val + 1;
    for (int i = 0; i < n; ++i) {
        random_numbers.push_back(lcg.next() % range + min_val);
    }
    
    int64_t max_sum = std::numeric_limits<int64_t>::min();
    for (int i = 0; i < n; ++i) {
        int64_t current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
 

In [41]:
# Compile C++ and run the executable

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized_practice03_hard_gpt.cpp
!./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.001839 seconds


In [42]:
# Compile C++ and run the executable

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized_practice03_hard_claude.cpp
!./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.487203 seconds


# Gradio UI

## GPT Code

In [18]:
gpt_model = 'gpt-4o-mini'

In [19]:
def ui_gpt(python):
    response = openai.chat.completions.create(model = gpt_model, messages = messages_for(python),stream = True)
    result = ''
    display_handle = display(Markdown, display_id = True)
    for chunk in response:
        result += chunk.choices[0].delta.content or ''
        result = result.replace('```cpp\n','').replace('```','')
        yield result

## Claude Code

In [20]:
def ui_claude(python):
    response = claude.messages.stream(
        messages = [{'role':'user','content':user_prompt_for(python)}],
        system = system_message,
        max_tokens = 2000,
        model = claude_model
    )
    result = ''
    with response as stream:
        for text in stream.text_stream:
            result += text
            yield result.replace('```cpp\n','').replace('```','')

## Gemini Code

In [21]:
def ui_gemini(python):
    gemini = google.generativeai.GenerativeModel(
        system_instruction = system_message,
        model_name = gemini_model
    )
    response = gemini.generate_content(user_prompt_for(python), stream = True)
    result = ''
    for chunk in response:
        if chunk.text:
            result += chunk.text
            yield result

In [22]:
def optimize(python, model):
    if model == 'GPT':
        result =  ui_gpt(python)
    elif model == 'Claude':
        result = ui_claude(python)
    else:
        raise ValueError('Invalid Model')
    for chunk in result:
        yield chunk

### UI

In [ ]:
with gr.Blocks() as app:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=20)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

app.launch(debug = True, share = True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://11d82b6af3c06ca613.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Improved Gradio

In [23]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code,globals())
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [24]:
def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [25]:
css = """
.python {background-color: #3B82F6;}
.cpp {background-color: #32CD32;}
"""

In [51]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


IPython.core.display.Markdown

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 15.058133 seconds
Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 15.058606 seconds
this is a string


# Open Source Models (Hugging Face)

In [26]:
from huggingface_hub import login, InferenceClient

In [27]:
from transformers import AutoTokenizer

In [28]:
hf_token = os.getenv('HF_API_KEY')
login(hf_token, add_to_git_credential = True)

In [1]:
code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
CODE_QWEN_URL = 'https://pqflffzn7hu8y84g.us-east-1.aws.endpoints.huggingface.cloud'

code_llama = 'codellama/CodeLlama-70b-Instruct-hf'
CODE_LLAMA_URL = 'https://qzziiwlqibu37ekc.us-east-2.aws.endpoints.huggingface.cloud'

code_star_coder = 'bigcode/starcoder2-15b'
CODE_STAR_CODER_URL = 'https://aetslxqm2vjlozdw.us-east-1.aws.endpoints.huggingface.cloud'

code_gemma = 'google/codegemma-7b-it'
CODE_GEMMA_URL = 'https://l77mnbo5de1yfudo.us-east-1.aws.endpoints.huggingface.cloud'

### Code Qwen

In [147]:
tokenizer = AutoTokenizer.from_pretrained(code_qwen,trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
messages = messages_for(pi)
text = tokenizer.apply_chat_template(messages , tokenize = False, add_generation_prompt = True)

client = InferenceClient(CODE_QWEN_URL,token = hf_token)
response = client.text_generation(text,stream = False, details = False,max_new_tokens = 3000)
print(response)

<|im_start|>system
You are an assistant that reimplements Python code in high performance C++ for an M4 Mac Pro. Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. The C++ response needs to produce an identical output in the fastest possible time.Keep implementations of random number generators identical so that results match perfectly.<|im_end|>
<|im_start|>user
Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. Respond only with C++ code; do not explain your work other than a few comments. Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.


import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

In [30]:
def stream_code_qwen(python):
    tokenizer = AutoTokenizer.from_pretrained(code_qwen,trust_remote_code = True)
    tokenizer.pad_token = tokenizer.eos_token
    messages = messages_for(python)
    text = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt=True)
    client = InferenceClient(CODE_QWEN_URL, token = hf_token)
    response = client.text_generation(text,stream = False, details = False, max_new_tokens = 3000)
    return response

### Code Llama

In [37]:
tokenizer = AutoTokenizer.from_pretrained(code_llama,trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
messages = messages_for(pi)
text_llama = tokenizer.apply_chat_template(messages , tokenize = False, add_generation_prompt = True)

client = InferenceClient(CODE_LLAMA_URL, token = hf_token)
response = client.text_generation(text_llama,stream=True, details = True, max_new_tokens = 500,stop=["<|im_end|>", "</s>"])
for chunk in response:
    print(chunk.token.text , end = '')


```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; i++) {
        int j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << "Result: " << std::setprecision(12) << result << std::endl;
    std::cout << "Execution Time: " << std::setprecision(6) << std::chrono::duration_cast<std::chrono::duration<double>>(end_time - start_time).count() << " seconds" << std::endl;

    return 0;
}
```

This C++ code is a direct translation of the Python code, with the exception of the `time` module, which is replaced with the `chrono` library for measuring execution time. The 

In [38]:
def stream_code_llama(python):
    tokenizer = AutoTokenizer.from_pretrained(code_llama, trust_remote_code = True)
    tokenizer.pad_token = tokenizer.eos_token
    messages = messages_for(python)
    text = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = True)

    client = InferenceClient(CODE_LLAMA_URL, token = hf_token)
    response = client.text_generation(text, stream = True, details = True, max_new_tokens = 1000,stop=["<|im_end|>", "```", "</s>"])
    
    result = ''
    for chunk in response:
        result += chunk.token.text
        yield result

### Star Coder

In [120]:
client = InferenceClient(CODE_STAR_CODER_URL, token = hf_token)
text = (
        "### Instruction:\n"
        "You are an expert C++ programmer.\n"
        "Rewrite the following Python code into the fastest equivalent C++ implementation.\n"
        "Respond with **only C++ code** and do not repeat this instruction.\n\n"
        f"{python}\n\n"
        "### Response:\n"
    )
response = client.text_generation(text, stream = True, details = True, max_new_tokens = 500,stop=["###", "<|im_end|>", "</s>"])
for chunk in response:
    print(chunk.token.text, end = '')


```cpp
#include <iostream>
#include <vector>
#include <algorithm>

using namespace std;

int main() {
    int n;
    cin >> n;
    vector<int> a(n);
    for (int i = 0; i < n; i++) {
        cin >> a[i];
    }
    sort(a.begin(), a.end());
    for (int i = 0; i < n; i++) {
        cout << a[i] << " ";
    }
    cout << endl;
    return 0;
}
```

###

In [121]:
def stream_star_coder(python):
    tokenizer = AutoTokenizer.from_pretrained(code_star_coder,trust_remote_code = True)
    tokenizer.pad_token = tokenizer.eos_token
    messages = messages_for(python)
    text = (
        "### Instruction:\n"
        "You are an expert C++ programmer.\n"
        "Rewrite the following Python code into the fastest equivalent C++ implementation.\n"
        "Respond with **only C++ code** and do not repeat this instruction.\n\n"
        f"{python}\n\n"
        "### Response:\n"
    )

    client = InferenceClient(CODE_STAR_CODER_URL, token = hf_token)
    response = client.text_generation(text, stream = True, details = True, max_new_tokens = 1500,stop=["###", "<|im_end|>", "```", "</s>"])
    result = ''
    for chunk in response:
        result += chunk.token.text
        yield result

### Code Gemma

In [137]:
tokenizer = AutoTokenizer.from_pretrained(code_gemma,trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
messages = [{'role':'user','content':user_prompt_for(pi)}]
text_gemma = tokenizer.apply_chat_template(messages , tokenize = False, add_generation_prompt = True)

client = InferenceClient(CODE_GEMMA_URL, token = hf_token)
response = client.text_generation(text_gemma, stream = True, details = True, max_new_tokens = 800)
for chunk in response:
    print(chunk.token.text, end = '')

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

using namespace std::chrono;

double calculate(int iterations, int param1, int param2) {
  double result = 1.0;
  for (int i = 1; i <= iterations; i++) {
    int j = i * param1 - param2;
    result -= (1.0 / j);
    j = i * param1 + param2;
    result += (1.0 / j);
  }
  return result;
}

int main() {
  auto start_time = high_resolution_clock::now();
  double result = calculate(100000000, 4, 1) * 4;
  auto end_time = high_resolution_clock::now();

  auto duration = duration_cast<microseconds>(end_time - start_time);

  cout << "Result: " << fixed << setprecision(12) << result << endl;
  cout << "Execution Time: " << fixed << setprecision(6) << duration.count() / 1000000.0 << " seconds" << endl;

  return 0;
}
```<eos>

In [138]:
def stream_code_gemma(python):
    tokenizer = AutoTokenizer.from_pretrained(code_gemma,trust_remote_code = True)
    tokenizer.pad_token = tokenizer.eos_token
    messages = [{'role':'user','content':user_prompt_for(python)}]
    text_gemma = tokenizer.apply_chat_template(messages , tokenize = False, add_generation_prompt = True)
    
    client = InferenceClient(CODE_GEMMA_URL, token = hf_token)
    response = client.text_generation(text_gemma, stream = True, details = True, max_new_tokens = 1500)
    result = ''
    for chunk in response:
        result += chunk.token.text
        yield result

In [39]:
def optimize_final(python, model):
    if model == "GPT":
        for chunk in ui_gpt(python):  
            yield chunk
    elif model == "Claude":
        for chunk in ui_claude(python):  
            yield chunk
    elif model == 'Gemini':
        for chunk in ui_gemini(python):
            yield chunk
    elif model == "CodeQwen":
        result = stream_code_qwen(python)
        yield result
    elif model == 'CodeLlama':
        for chunk in stream_code_llama(python):
            yield chunk
    elif model == 'StarCoder':
        for chunk in stream_star_coder(python):
            yield chunk
    elif model =='Gemma':
        for chunk in stream_code_gemma(python):
            yield chunk
    else:
        raise ValueError("Unknown model")

In [40]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude",'Gemini',"CodeQwen",'CodeLlama','StarCoder','Gemma'], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize_final, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
